In [36]:
import gzip
import shutil
from pathlib import Path
from datetime import datetime
import re, csv


def QScore_Cal(ASCII):
    list_Qscore = []
    list_ASCII = []
    Dict_Qscore = {}
    length_ASCII = len(ASCII)
    for each_ASCII in ASCII:
        each_Qscore = ord(each_ASCII)-33
        list_Qscore.append(each_Qscore)
        list_ASCII.append(each_ASCII)
    sum_Qscore = sum(list_Qscore)
    average_Qscore = sum_Qscore/length_ASCII
    return average_Qscore

def ReadLength(gzfile):
    output = []    
    outputtime = datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    with open('output-' + outputtime + '.tsv','w') as tableOut:
        writer = csv.writer(tableOut, delimiter = "\t")
        writer.writerow(['Read_ID', 'Barcode', 'Read_length', 'Average_Qscore'])
        
    with gzip.open(gzfile, 'rb') as filein:
        with open('processfile.fastq', 'wb') as fileout:
            shutil.copyfileobj(filein, fileout)
            path = Path('processfile.fastq')
            with open(path, 'r') as readfile:
                lines = readfile.read()
                for data in re.finditer (r'@(?P<Read_ID>\w*).*barcode=(?P<Read_barcode>\w*).*\n.*\n\+\n(?P<Read_Qscore>.*)', lines):
                    readID = data.group('Read_ID')
                    readBarcode = data.group('Read_barcode')
                    readQscore = data.group('Read_Qscore')
                    readlength = str(len(readQscore))
                    readscore = data.group('Read_Qscore')
                    dataoutput = (readID, readBarcode, readlength, QScore_Cal(readQscore))
                    outputlist = list(dataoutput)

                    with open('output-' + outputtime + '.tsv','a') as tableOut:
                            writer = csv.writer(tableOut, delimiter = "\t")
                            for add in outputlist:
                                output.append(add)
                            writer.writerow(outputlist)
                            
    path.unlink()
    
    print ("Read length analysis complete")
    
    
# ReadLength("ont.exp2.fastq.gz")
ReadLength("sampleNew.fastq.gz")

Read length analysis complete
